In [150]:
#https://www.kaggle.com/code/nataliadelarosa/sql-case-study-1/notebook 
#https://8weeksqlchallenge.com/case-study-1/

import pandas as pd
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)
import sqlite3
print('complete...')

complete...


In [151]:
try:
    conn = sqlite3.connect('databases/danny_diner.db') # # Creates connection to existing db or creates an empty db
    print('Connected successfully...')  
except Exception as e:
    print('Error during connection:', str(e))
else: 
    c = conn.cursor() # # Cursor allows SQL commands to be executed. 
    c.execute( '''
    CREATE TABLE sales (
          "customer_id" VARCHAR(1),
          "order_date" DATE,
          "product_id" INTEGER
        ); ''')

    c.execute('''
        INSERT INTO sales
          ("customer_id", "order_date", "product_id")
        VALUES
          ('A', '2021-01-01', '1'),
          ('A', '2021-01-01', '2'),
          ('A', '2021-01-07', '2'),
          ('A', '2021-01-10', '3'),
          ('A', '2021-01-11', '3'),
          ('A', '2021-01-11', '3'),
          ('B', '2021-01-01', '2'),
          ('B', '2021-01-02', '2'),
          ('B', '2021-01-04', '1'),
          ('B', '2021-01-11', '1'),
          ('B', '2021-01-16', '3'),
          ('B', '2021-02-01', '3'),
          ('C', '2021-01-01', '3'),
          ('C', '2021-01-01', '3'),
          ('C', '2021-01-07', '3'); ''')

    c.execute('''
        CREATE TABLE menu (
          "product_id" INTEGER,
          "product_name" VARCHAR(5),
          "price" INTEGER
        );''')

    c.execute('''
        INSERT INTO menu
          ("product_id", "product_name", "price")
        VALUES
          ('1', 'sushi', '10'),
          ('2', 'curry', '15'),
          ('3', 'ramen', '12'); ''')

    c.execute('''     
        CREATE TABLE members (
          "customer_id" VARCHAR(1),
          "join_date" DATE
        );''')

    c.execute('''
        INSERT INTO members
          ("customer_id", "join_date")
        VALUES
          ('A', '2021-01-07'),
          ('B', '2021-01-09'); ''')
    conn.commit()
    print('complete...')

Connected successfully...


OperationalError: table sales already exists

In [152]:
#1. What is the total amount each customer spent at the restaurant? 

out1 = pd.read_sql(''' 
SELECT s.customer_id, SUM(price) AS total_price  
FROM sales s 
JOIN menu m ON s.product_id = m.product_id
GROUP BY s.customer_id;
    
''', conn)
out1

,customer_id,total_price
0,A,76
1,B,74
2,C,36


In [153]:
# 2. How many days has each customer visited the restaurant?
out1 = pd.read_sql(''' 
SELECT customer_id, COUNT(DISTINCT order_date)  
FROM sales  
GROUP BY customer_id;
''', conn)
out1

,customer_id,COUNT(DISTINCT order_date)
0,A,4
1,B,6
2,C,2


In [154]:
#3. What was the first item from the menu purchased by each customer?

#approach1 : using windows functions
out1 = pd.read_sql(''' 
SELECT * 
FROM 
    (SELECT s.customer_id, m.product_name, RANK() OVER(PARTITION BY s.customer_id ORDER BY s.order_date) AS rnk
    FROM sales s 
    JOIN menu m ON s.product_id = m.product_id) x
WHERE x.rnk = 1;

    
''', conn)
out1

#approach2 : using joins and group by
# first_item = pd.read_sql('''
# SELECT s.customer_id
#     , m.product_name
# FROM sales s
# JOIN menu m USING(product_id)
# WHERE s.order_date IN (
#     SELECT MIN(order_date)
#     FROM sales
#     GROUP BY customer_id
   
#     );''', conn)
# first_item

,customer_id,product_name,rnk
0,A,sushi,1
1,A,curry,1
2,B,curry,1
3,C,ramen,1
4,C,ramen,1


In [155]:
#4. What is the most purchased item on the menu and how many times was it purchased by all customers?
out1 = pd.read_sql(''' 
SELECT m.product_name, COUNT(s.product_id) AS total_purchases
FROM sales s 
JOIN menu m ON s.product_id = m.product_id
GROUP BY s.product_id
ORDER BY COUNT(s.product_id) DESC 
LIMIT 1;


''', conn)
out1

,product_name,total_purchases
0,ramen,8


In [156]:
#5.Which item was the most popular for each customer?
# out1 = pd.read_sql(''' 
# select  m.product_name, count(s.product_id) as total_purchases, s.customer_id from sales s join menu m on s.product_id = m.product_id
# group by s.customer_id,s.product_id
# order by count(s.product_id) desc 

# ''', conn)
# out1

out1 = pd.read_sql(''' 
WITH counts (customer_id, product_id, total_purchases) AS 
    (SELECT customer_id, product_id, COUNT(product_id) 
    FROM sales 
    GROUP BY customer_id, product_id ) 
  
SELECT *
FROM ( 
SELECT customer_id, product_id, total_purchases, RANK() OVER(PARTITION BY customer_id ORDER BY total_purchases DESC) AS rank
FROM counts )
WHERE rank = 1;


''', conn)
out1


,customer_id,product_id,total_purchases,rank
0,A,3,3,1
1,B,1,2,1
2,B,2,2,1
3,B,3,2,1
4,C,3,3,1


In [157]:
#6.Which item was purchased first by the customer after they became a member?
out1 = pd.read_sql(''' 
SELECT customer_id, order_date, product_id
FROM ( 
SELECT s.customer_id, s.order_date, s.product_id, RANK() OVER(PARTITION BY s.customer_id ORDER BY s.order_date) AS rank
FROM sales s
WHERE s.order_date >= 
    (SELECT m.join_date
    FROM members m
    WHERE s.customer_id = m.customer_id) )
WHERE rank = 1;

''', conn)
out1

,customer_id,order_date,product_id
0,A,2021-01-07,2
1,B,2021-01-11,1


In [158]:
#7. Which item was purchased just before the customer became a member?

out1 = pd.read_sql(''' 
SELECT * 
FROM sales s 
JOIN members m USING(customer_id)
WHERE s.order_date < m.join_date;
''', conn)
out1

,customer_id,order_date,product_id,join_date
0,A,2021-01-01,1,2021-01-07
1,A,2021-01-01,2,2021-01-07
2,B,2021-01-01,2,2021-01-09
3,B,2021-01-02,2,2021-01-09
4,B,2021-01-04,1,2021-01-09


In [159]:
#8. What is the total items and amount spent for each member before they became a member? 
out1 = pd.read_sql(''' 
select 
  s.customer_id, 
  count(s.product_id) as num_items, 
  sum(m2.price) as total_price 
from 
  sales s 
  join members m1 on s.customer_id = m1.customer_id 
  join menu m2 on s.product_id = m2.product_id 
where 
  s.order_date < m1.join_date 
group by 
  s.customer_id
''', conn)
out1

,customer_id,num_items,total_price
0,A,2,25
1,B,3,40


In [160]:
#9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

out1 = pd.read_sql(''' 
WITH points_t AS
    (SELECT s.customer_id, 
        CASE
        WHEN m2.product_name = "sushi" THEN m2.price * 20
        ELSE m2.price * 10
        END AS points
    FROM sales s JOIN menu m2 on s.product_id = m2.product_id
    )

SELECT customer_id, sum(points) as total_points
FROM points_t
GROUP BY  customer_id

''', conn)
out1


,customer_id,total_points
0,A,860
1,B,940
2,C,360


In [161]:
#10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?
out1 = pd.read_sql(''' 
WITH points_t AS
    (SELECT s.customer_id, m2.join_date, s.order_date, m2.price, (julianday(s.order_date) - julianday(m2.join_date)) as diff, 
        CASE
        WHEN m2.product_name = "sushi" THEN m2.price * 20 
        WHEN m2.product_name != "sushi" AND (s.order_date BETWEEN  m2.join_date AND DATE(m2.join_date,'+6 day')) THEN m2.price * 20
        ELSE m2.price * 10
        END AS points
    FROM sales s JOIN menu m2 on s.product_id = m2.product_id JOIN members m2 USING(customer_id)
    )

SELECT customer_id, sum(points) as total_points
FROM points_t
GROUP BY  customer_id


''', conn)
out1

,customer_id,total_points
0,A,1370
1,B,940
